In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-24 18:25:25--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-24 18:25:25 (11.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [33]:
import minsearch

In [34]:
import json

In [35]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [36]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [37]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [38]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)

In [39]:
q = 'the course has already started, can I still enroll?'

In [40]:
index.fit(documents)

In [41]:
from openai import OpenAI

In [42]:
client = OpenAI()

In [43]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It depends on the specific policies of the course provider or institution offering the course. Here are a few steps you can take:\n\n1. **Check the Enrollment Deadline:** Many courses have a specific deadline for enrollment. Look up the course's details online or in the course catalog to see if the deadline has passed.\n\n2. **Contact the Instructor:** If the deadline has passed, reach out to the course instructor or professor. Often, instructors have the discretion to allow late enrollments, especially if the course has just started.\n\n3. **Contact the Administration:** If you can't get in touch with the instructor, try contacting the department or registrar's office. They can provide information about late enrollment policies and any potential fees associated with late registration.\n\n4. **Self-Study:** If enrollment is not possible, consider asking if you can audit the course or access the materials for self-study. This might not give you official credit but will allow you to lea

In [44]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [45]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [46]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [47]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [49]:
print(rag(query))

To run Kafka, follow the instructions specific to the type of Kafka application you are working with. Here are the details for running different types of Kafka applications:

### Java Kafka:
To run a Java Kafka producer/consumer/kstreams/etc. in the terminal, navigate to your project directory and use the following command:
```sh
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

### Python Kafka:
If you encounter the "Module 'kafka' not found" error when trying to run `producer.py`, you should create a virtual environment and install the necessary packages within that environment. Here's how:

1. **Create a Virtual Environment and Install Packages (run only once)**:
    ```sh
    python -m venv env
    source env/bin/activate
    pip install -r ../requirements.txt
    ```

2. **Activate the Virtual Environment (you'll need to run this every time you need the virtual env)**:
    ```sh
    source env/bin/activate
    ```

3. **Deactivate

In [50]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course even after it has already started. You are still eligible to submit the homework assignments, but be cautious not to miss the deadlines for the final projects.'

In [51]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [53]:
from elasticsearch import Elasticsearch

In [54]:
es_client = Elasticsearch('http://localhost:9200')

In [55]:
es_client.info()

ObjectApiResponse({'name': '382e7b04754d', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'IpX5G5XRQTK9ye31nnbbvQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [59]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [60]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [63]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:23<00:00, 40.14it/s]


In [65]:
query = 'the course has already started, can I still enroll?'

In [73]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [74]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [75]:
rag(query)

"Yes, you can still enroll in the course even after it has started. You are eligible to submit the homeworks even if you haven't registered. However, be mindful of the deadlines for submitting the final projects, so ensure you manage your time effectively and don't leave everything for the last minute."